In [658]:
from constants import *
class Tile():
    def is_blocking(self) -> bool:
        return False

    def damage(self) -> int:
        return 0

    def get_id(self) -> str:
        return ABSTRACT_TILE

    def __str__(self) -> str:
        return self.get_id()

    def __repr__(self) -> str:
        return self.__class__.__name__ + "()"

In [659]:
t = Tile()
t.is_blocking()

False

In [660]:
t.damage()

0

In [661]:
t.get_id()

'AT'

In [662]:
str(t)

'AT'

In [663]:
t

Tile()

In [664]:
class Wall(Tile):
    def is_blocking(self) -> bool:
        return True

    def get_id(self) -> str:
        return WALL

In [665]:
wall = Wall()
wall

Wall()

In [666]:
wall.is_blocking()

True

In [667]:
wall.get_id()

'#'

In [668]:
str(wall)

'#'

In [669]:
class Entity:
    def __init__(self, position: tuple[int, int]) -> None:
        self.position = position

    def get_position(self) -> tuple[int, int]:
        return self.position

    def get_name(self) -> str:
        return self.__class__.__name__

    def get_id(self) -> str:
        return 'E'

    def __str__(self) -> str:
        return self.get_id()

    def __repr__(self) -> str:
        return self.get_name() + "(" + str(self.position) + ")"


In [670]:
entity = Entity((2, 3))
entity

Entity((2, 3))

In [671]:
entity.get_position()

(2, 3)

In [672]:
entity.get_name()

'Entity'

In [673]:
entity.get_id()

'E'

In [674]:
str(entity)

'E'

In [675]:
class DynamicEntity(Entity):
    def __init__(self, position: tuple[int, int]) -> None:
        super().__init__(position)

    def set_position(self, new_position: tuple[int, int]) -> None:
        self.position = new_position

    def get_id(self) -> str:
        return DYNAMIC_ENTITY

In [676]:
dynamic_entity = DynamicEntity((1, 1))
dynamic_entity

DynamicEntity((1, 1))

In [677]:
dynamic_entity.get_position()

(1, 1)

In [678]:
dynamic_entity.set_position((2, 3))
dynamic_entity.get_position()

(2, 3)

In [679]:
dynamic_entity.get_id()

'DE'

In [680]:
dynamic_entity

DynamicEntity((2, 3))

In [681]:
from typing import Optional

class Player(DynamicEntity):
    def __init__(self, position: tuple[int, int]) -> None:
        super().__init__(position)
        self.health = MAX_HEALTH
        self.hunger = 0
        self.thirst = 0
        self.inventory = Inventory()

    def get_id(self) -> str:
        return PLAYER

    def get_hunger(self) -> int:
        return self.hunger

    def get_thirst(self) -> int:
        return self.thirst

    def get_health(self) -> int:
        return self.health

    def change_hunger(self, amount: int) -> None:
        self.hunger += amount
        if self.hunger < 0:
            self.hunger = 0
        elif self.hunger > MAX_HUNGER:
            self.hunger = MAX_HUNGER

    def change_thirst(self, amount: int) -> None:
        self.thirst += amount
        if self.thirst < 0:
            self.thirst = 0
        elif self.thirst > MAX_THIRST:
            self.thirst = MAX_THIRST

    def change_health(self, amount: int) -> None:
        self.health += amount
        if self.health < 0:
            self.health = 0
        elif self.health > MAX_HEALTH:
            self.health = MAX_HEALTH

    def get_inventory(self):
        return self.inventory

    def add_item(self, item) -> None:
        self.inventory.add_item(item)


class Item(Entity):
    def __init__(self, position: tuple[int, int]) -> None:
        super().__init__(position)

    def get_id(self) -> str:
        return ITEM

    def apply(self, player: Player) -> None:
        player.add_item(self)


class Potion(Item):
    def __init__(self, position: tuple[int, int]) -> None:
        super().__init__(position)
        self.health_change = POTION_AMOUNT

    def get_id(self) -> str:
        return POTION

    def apply(self, player: Player) -> None:
        player.change_health(self.health_change)


class Coin(Item):
    def __init__(self, position: tuple[int, int]) -> None:
        super().__init__(position)

    def get_id(self) -> str:
        return COIN

    # def apply(self, player: Player) -> None:
    #     player.add_item(self)


class Water(Item):
    def __init__(self, position: tuple[int, int]) -> None:
        super().__init__(position)
        self.thirst_change = WATER_AMOUNT

    def get_id(self) -> str:
        return WATER

    def apply(self, player: Player) -> None:
        player.change_thirst(self.thirst_change)


class Food(Item):
    def __init__(self, position: tuple[int, int]) -> None:
        super().__init__(position)

    def get_id(self) -> str:
        return FOOD


class Apple(Food):
    def __init__(self, position: tuple[int, int]) -> None:
        super().__init__(position)
        self.hunger_change = APPLE_AMOUNT

    def get_id(self) -> str:
        return APPLE

    def apply(self, player: Player) -> None:
        player.change_hunger(self.hunger_change)


class Honey(Food):
    def __init__(self, position: tuple[int, int]) -> None:
        super().__init__(position)
        self.hunger_change = HONEY_AMOUNT

    def get_id(self) -> str:
        return HONEY

    def apply(self, player: Player) -> None:
        player.change_hunger(self.hunger_change)


class Inventory:
    def __init__(self, initial_items: Optional[list[Item]] = None) -> None:
        self.items = {}
        if initial_items is not None:
            for item in initial_items:
                self.add_item(item)

    def add_item(self, item: Item) -> None:
        if item.get_name() not in self.items:
            self.items[item.get_name()] = [item]
        else:
            self.items[item.get_name()].append(item)

    def get_items(self) -> dict[str, list[Item]]:
        return self.items

    def remove_item(self, item_name: str) -> Optional[Item]:
        if item_name in self.items:
            item = self.items[item_name].pop(0)
            if len(self.items[item_name]) == 0:
                del self.items[item_name]
            return item
        return None

    def __str__(self) -> str:
        rtn = ""
        for item in self.items:
            rtn += f"{item}: {len(self.items[item])}\n"
        return rtn


    def __repr__(self) -> str:
        return str(self.items)

In [682]:
player = Player((2, 3))
player

Player((2, 3))

In [683]:
player.get_thirst()

0

In [684]:
player.get_hunger()

0

In [685]:
player.get_health()

100

In [686]:
player.change_hunger(4)

In [687]:
player.change_thirst(3)

In [688]:
player.change_health(-34)

In [689]:
player.get_hunger()

4

In [690]:
player.get_thirst()

3

In [691]:
player.get_health()

66

In [692]:
player.get_inventory().get_items()

{}

In [693]:
player.add_item(Honey((2, 3)))
player.add_item(Honey((4, 4)))
player.add_item(Water((1, 1)))

In [694]:
player.get_inventory().get_items()

{'Honey': [Honey((2, 3)), Honey((4, 4))], 'Water': [Water((1, 1))]}

In [695]:
player

Player((2, 3))

In [696]:
player = Player((1, 1))
potion = Potion((1, 1))
player.change_health(-50)

In [697]:
player.get_health()

50

In [698]:
potion.apply(player)

In [699]:
player.get_health()

70

In [700]:
potion

Potion((1, 1))

In [701]:
player = Player((4, 4))
coin = Coin((4, 4))
print(player.get_health(), player.get_thirst(), player.get_hunger())

100 0 0


In [702]:
player.get_inventory().get_items()

{}

In [703]:
coin.apply(player)
print(player.get_health(), player.get_thirst(), player.get_hunger())

100 0 0


In [704]:
player.get_inventory().get_items()

{'Coin': [Coin((4, 4))]}

In [705]:
player = Player((1, 1))
player.change_thirst(8)
player.get_thirst()

8

In [706]:
water = Water((1, 1))
water.apply(player)
player.get_thirst()

3

In [707]:
water.get_id()

'W'

In [708]:
str(water)

'W'

In [709]:
water

Water((1, 1))

In [710]:
player = Player((1, 1))
player.change_hunger(7)
apple = Apple((1, 1))
honey = Honey((2, 3))
player.get_hunger()


7

In [711]:
apple.apply(player)
player.get_hunger()

6

In [712]:
honey.apply(player)
player.get_hunger()

1

In [713]:
apple.get_id() + honey.get_id()

'AH'

In [714]:
honey

Honey((2, 3))

In [715]:
apple

Apple((1, 1))

In [716]:
inventory = Inventory([Water((1, 2)), Honey((2, 3)), Water((3, 4))])
inventory

{'Water': [Water((1, 2)), Water((3, 4))], 'Honey': [Honey((2, 3))]}

In [717]:
inventory.get_items()

{'Water': [Water((1, 2)), Water((3, 4))], 'Honey': [Honey((2, 3))]}

In [718]:
inventory.add_item(Honey((3, 4)))
inventory.add_item(Coin((1, 1)))
inventory.get_items()

{'Water': [Water((1, 2)), Water((3, 4))],
 'Honey': [Honey((2, 3)), Honey((3, 4))],
 'Coin': [Coin((1, 1))]}

In [719]:
inventory.remove_item('Honey')

Honey((2, 3))

In [720]:
inventory.get_items()

{'Water': [Water((1, 2)), Water((3, 4))],
 'Honey': [Honey((3, 4))],
 'Coin': [Coin((1, 1))]}

In [721]:
inventory.remove_item('Coin')

Coin((1, 1))

In [722]:
inventory.get_items()

{'Water': [Water((1, 2)), Water((3, 4))], 'Honey': [Honey((3, 4))]}

In [723]:
print(inventory)

Water: 2
Honey: 1



In [724]:
class Empty(Tile):
    def is_blocking(self) -> bool:
        return False

    def get_id(self) -> str:
        return EMPTY


class Lava(Tile):
    def is_blocking(self) -> bool:
        return False

    def damage(self) -> int:
        return LAVA_DAMAGE

    def get_id(self) -> str:
        return LAVA


class Door(Tile):
    def __init__(self, is_blocked: bool = True, id: str = DOOR):
        self.is_blocked = is_blocked
        self.id = id

    def is_blocking(self) -> bool:
        return self.is_blocked

    def get_id(self) -> str:
        return self.id

    def unlock(self) -> None:
        self.is_blocked = False
        self.id = EMPTY

In [725]:

test_dic = {'#': Wall, "L": Lava}
s = "#CCLL"
d = []
for l in s: 
    if l in test_dic:
        d.append(test_dic[l]())
    else:
        d.append(Empty())
d

[Wall(), Empty(), Empty(), Lava(), Lava()]

In [726]:
class Maze:
    def __init__(self, dimensions: tuple[int, int]) -> None:
        self.dimensions = dimensions
        self.maze = []
        self.tile_pool = {LAVA: Lava, WALL: Wall, EMPTY: Empty, DOOR: Door}

    def get_dimensions(self) -> tuple[int, int]:
        return self.dimensions

    def add_row(self, row: str) -> None:
        assert len(row) == self.dimensions[1] and len(
            self.maze) < self.dimensions[0]
        real_row = []
        for tile in row:
            if tile in self.tile_pool:
                real_row.append(self.tile_pool[tile]())
            else:
                real_row.append(self.tile_pool[EMPTY]())
        self.maze.append(real_row)

    def get_tiles(self) -> list[list[Tile]]:
        return self.maze

    def unlock_door(self) -> None:
        for row in self.maze:
            for tile in row:
                if isinstance(tile, Door):
                    tile.unlock()

    def get_tile(self, position: tuple[int, int]) -> Tile:
        return self.maze[position[0]][position[1]]

    def __str__(self) -> str:
        rtn = ""
        for row in self.maze:
            for tile in row:
                rtn += str(tile)
            rtn += "\n"
        return rtn

    def __repr__(self) -> str:
        return self.__class__.__name__ + f"({self.dimensions})"

In [727]:
maze = Maze((5, 5))
maze.get_dimensions()

(5, 5)

In [728]:
maze.get_tiles()

[]

In [729]:
str(maze)

''

In [730]:
maze.add_row("#####")
maze.add_row("# C D")
maze.add_row("# C #")
maze.add_row("P C #")
maze.add_row("#####")

In [731]:
from pprint import pprint
pprint(maze.get_tiles())

[[Wall(), Wall(), Wall(), Wall(), Wall()],
 [Wall(), Empty(), Empty(), Empty(), Door()],
 [Wall(), Empty(), Empty(), Empty(), Wall()],
 [Empty(), Empty(), Empty(), Empty(), Wall()],
 [Wall(), Wall(), Wall(), Wall(), Wall()]]


In [732]:
str(maze)

'#####\n#   D\n#   #\n    #\n#####\n'

In [733]:
print(maze)

#####
#   D
#   #
    #
#####



In [734]:
maze

Maze((5, 5))

In [735]:
maze.get_tile((2, 3))

Empty()

In [736]:
maze.unlock_door()
print(maze)

#####
#    
#   #
    #
#####



In [737]:
pprint(maze.get_tiles())

[[Wall(), Wall(), Wall(), Wall(), Wall()],
 [Wall(), Empty(), Empty(), Empty(), Door()],
 [Wall(), Empty(), Empty(), Empty(), Wall()],
 [Empty(), Empty(), Empty(), Empty(), Wall()],
 [Wall(), Wall(), Wall(), Wall(), Wall()]]


In [738]:
class Level:
    def __init__(self, dimenstions: tuple[int, int]) -> None:
        self.dimensions = dimenstions
        self.maze = Maze(self.dimensions)
        self.items = {}
        self.item_pool = {'C': Coin, 'M': Potion,
                          'W': Water, 'A': Apple, 'H': Honey}
        self.player_start_position = None

    def get_maze(self) -> Maze:
        return self.maze
    
    def get_dimensions(self) -> tuple[int, int]:
        return self.dimensions

    def attempt_unlock_door(self) -> None:
        all_items = self.get_items()
        for item in all_items:
            if isinstance(all_items[item], Coin):
                return
        self.maze.unlock_door()

    def add_row(self, row: str) -> None:
        self.maze.add_row(row)
        row_no = len(self.maze.get_tiles()) - 1
        for col_no, element in enumerate(row):
            if element in self.item_pool:
                self.items[(row_no, col_no)] = self.item_pool[element](
                    (row_no, col_no))
            if element == PLAYER:
                self.add_player_start((row_no, col_no))
    
    def add_entity(self, position: tuple[int, int], entity_id: str) -> None:
        if entity_id in self.item_pool:
            self.items[position] = self.item_pool[entity_id](position)

    def get_items(self) -> dict[tuple[int, int], Item]:
        return self.items

    def remove_item(self, position: tuple[int, int]) -> None:
        if self.items[position].get_id() in self.item_pool:
            del self.items[position]

    def add_player_start(self, position: tuple[int, int]) -> None:
        self.player_start_position = position

    def get_player_start(self) -> Optional[tuple[int, int]]:
        return self.player_start_position

    def __str__(self) -> str:
        rtn = "Maze:\n"
        rtn += str(self.maze)
        rtn += f"Items:{self.items}\n"
        rtn += f"Player start: {self.get_player_start()}\n"
        return rtn

    def __repr__(self) -> str:
        return self.__class__.__name__ + f"({self.dimensions})"

In [739]:
level = Level((5, 5))
level

Level((5, 5))

In [740]:
level.get_maze()

Maze((5, 5))

In [741]:
level.get_maze().get_tiles()

[]

In [742]:
level.get_items()

{}

In [743]:
level.get_player_start()

In [744]:
level.get_dimensions()

(5, 5)

In [745]:
level.add_row("#####")
level.add_row("# C D")
level.add_row("# C #")
level.add_row("P C #")
level.add_row("#####")

In [746]:
print(level.get_maze())

#####
#   D
#   #
    #
#####



In [747]:
level.get_items()

{(1, 2): Coin((1, 2)), (2, 2): Coin((2, 2)), (3, 2): Coin((3, 2))}

In [748]:
level.get_player_start()

(3, 0)

In [749]:
level.add_entity((2, 3), 'M')

In [750]:
level.get_items()

{(1, 2): Coin((1, 2)),
 (2, 2): Coin((2, 2)),
 (3, 2): Coin((3, 2)),
 (2, 3): Potion((2, 3))}

In [751]:
level.attempt_unlock_door()

In [752]:
print(level.get_maze())

#####
#   D
#   #
    #
#####



In [753]:
level.remove_item((1, 2))
level.remove_item((2, 2))
level.remove_item((3, 2))
level.get_items()

{(2, 3): Potion((2, 3))}

In [754]:
level.attempt_unlock_door()
print(level.get_maze())

#####
#    
#   #
    #
#####



In [755]:
from pprint import pprint
pprint(level.get_maze().get_tiles())

[[Wall(), Wall(), Wall(), Wall(), Wall()],
 [Wall(), Empty(), Empty(), Empty(), Door()],
 [Wall(), Empty(), Empty(), Empty(), Wall()],
 [Empty(), Empty(), Empty(), Empty(), Wall()],
 [Wall(), Wall(), Wall(), Wall(), Wall()]]


In [756]:
print(level)

Maze:
#####
#    
#   #
    #
#####
Items:{(2, 3): Potion((2, 3))}
Player start: (3, 0)

